In [1]:
import pandas as pd
import numpy as np

In [2]:
X = pd.read_csv('/home/shubham/diabetes/diabetes.csv')
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Pedigree,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
display(X.columns)
X.dtypes

Index([u'Pregnancies', u'Glucose', u'BloodPressure', u'SkinThickness',
       u'Insulin', u'BMI', u'Pedigree', u'Age', u'Outcome'],
      dtype='object')

Pregnancies        int64
Glucose            int64
BloodPressure      int64
SkinThickness      int64
Insulin            int64
BMI              float64
Pedigree         float64
Age                int64
Outcome            int64
dtype: object

In [4]:
class_counts = X.groupby('Outcome').size()
class_counts

Outcome
0    500
1    268
dtype: int64

In [5]:
y = X.Outcome
X = X.drop('Outcome',axis=1)
display(X.head())
y.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Pedigree,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [6]:
from sklearn import preprocessing
T = preprocessing.scale(X)

from sklearn.decomposition import PCA
pca = PCA(n_components = 4)
X_pca = pca.fit_transform(T)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.33)

In [13]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
score

0.6889763779527559

In [9]:
from sklearn.manifold import Isomap
best_score,K,L,I,J = 0,0,0,0,0
for k in range(2, 6):
    for l in range(3, 7):
        iso = Isomap(n_neighbors = k, n_components = l)
        X_iso = iso.fit_transform(T)
        X_train, X_test, y_train, y_test = train_test_split(X_iso, y, test_size = 0.33)
    for i in np.arange(start = 0.05, stop = 2.05, step = 0.05):
                for j in np.arange(start = 0.001, stop = 0.101, step = 0.001):
                    model = SVC(C = i, gamma = j)
                    model.fit(X_train, y_train)
                    score = model.score(X_test, y_test)
                    if score > best_score:
                        best_score,K,L,I,J = score,k,l,i,j
                        best_C = model.C
                        best_gamma = model.gamma
best_score

0.77165354330708658

In [10]:
iso = Isomap(n_neighbors=K,n_components=L)
X_iso = iso.fit(T)
T_reduced = X_iso.transform(T)
X_train, X_test, y_train, y_test = train_test_split(T_reduced, y, test_size = 0.33)
model=SVC(C=I,gamma=J, probability = True)
model.fit(X_train,y_train)

SVC(C=1.2000000000000002, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.049000000000000002,
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [11]:
var = [[6,148,72,35,0,33.6,0.627,50]]
var_pre = X_iso.transform(var)
ans = model.predict_proba(var_pre)
print 'probability of having diabetes is: ' + str(ans[0][1])
print 'probability of not having diabetes is: ' + str(ans[0][0])

probability of having diabetes is: 0.446667776021
probability of not having diabetes is: 0.553332223979
